# 🔹 모듈 A: RNN 기반 시계열 예측 (Netflix 주가 예측)

## 📚 학습 목표
이 모듈에서는 순환 신경망(RNN)과 그 변형인 LSTM, GRU를 사용하여 시계열 데이터를 예측하는 방법을 학습합니다.

**주요 학습 내용:**
- RNN, LSTM, GRU의 내부 구조와 수학적 원리
- 그래디언트 소실/폭주 문제와 해결 방법
- 합성 데이터와 실제 주가 데이터를 이용한 실험
- 하이퍼파라미터 튜닝과 성능 평가

**대상 학습자:** 석사 수준의 딥러닝 학습자  
**예상 학습 시간:** 4-6시간


---
# 1️⃣ 이론 파트

## 1.1 순환 신경망(RNN)의 기본 구조

### 왜 RNN이 필요한가?

일반적인 피드포워드 신경망은 각 입력을 독립적으로 처리합니다. 하지만 시계열 데이터나 자연어와 같이 **순서가 중요한 데이터**에서는 이전 정보를 기억해야 합니다.

RNN은 **순환 구조**를 통해 이전 시점의 정보를 현재 시점으로 전달할 수 있습니다.

### RNN의 수학적 정의

RNN의 기본 순전파 수식은 다음과 같습니다:

$$h_t = \tanh(W_h h_{t-1} + W_x x_t + b_h)$$

$$y_t = W_y h_t + b_y$$

**기호 설명:**
- $x_t$: 시점 $t$에서의 입력 벡터
- $h_t$: 시점 $t$에서의 은닉 상태 (hidden state)
- $h_{t-1}$: 이전 시점의 은닉 상태
- $y_t$: 시점 $t$에서의 출력
- $W_h$: 은닉 상태 가중치 행렬
- $W_x$: 입력 가중치 행렬
- $W_y$: 출력 가중치 행렬
- $b_h, b_y$: 편향(bias)
- $\tanh$: 하이퍼볼릭 탄젠트 활성화 함수

### RNN의 동작 원리

1. **초기 은닉 상태** $h_0$는 보통 0으로 초기화
2. 각 시점 $t$마다:
   - 현재 입력 $x_t$와 이전 은닉 상태 $h_{t-1}$을 결합
   - 가중치 행렬을 곱하고 활성화 함수 적용
   - 새로운 은닉 상태 $h_t$ 생성
   - 필요시 출력 $y_t$ 생성

### 간단한 수치 예제

RNN의 동작을 이해하기 위한 간단한 예제:

**가정:**
- 입력 차원: 1
- 은닉 상태 차원: 2
- $W_h = \begin{bmatrix} 0.5 & 0.2 \\ 0.3 & 0.4 \end{bmatrix}$, $W_x = \begin{bmatrix} 0.6 \\ 0.7 \end{bmatrix}$, $b_h = \begin{bmatrix} 0.1 \\ 0.1 \end{bmatrix}$

**계산:**
- 시점 t=0: $h_0 = [0, 0]$, $x_0 = 1.0$
- $h_1 = \tanh(W_h h_0 + W_x x_0 + b_h) = \tanh([0.6 \times 1.0 + 0.1, 0.7 \times 1.0 + 0.1]) = \tanh([0.7, 0.8])$
- $h_1 ≈ [0.604, 0.664]$

이런 식으로 시점마다 은닉 상태가 업데이트되며 이전 정보를 보존합니다.


## 1.2 그래디언트 소실/폭주 문제

### 문제의 원인

RNN을 역전파(BPTT: Backpropagation Through Time)로 학습할 때, 그래디언트는 시간 축을 따라 역으로 전파됩니다:

$$\frac{\partial L}{\partial h_{t-k}} = \frac{\partial L}{\partial h_t} \prod_{i=t-k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}}$$

여기서 $\frac{\partial h_i}{\partial h_{i-1}}$는 주로 $W_h$와 활성화 함수의 미분을 포함합니다.

**문제:**
- 만약 이 값이 **1보다 작으면**: 여러 번 곱해질수록 0에 가까워짐 → **그래디언트 소실**
- 만약 이 값이 **1보다 크면**: 여러 번 곱해질수록 무한대로 발산 → **그래디언트 폭주**

### 수치적 예시

그래디언트 소실의 예:
- $\frac{\partial h_i}{\partial h_{i-1}} = 0.5$ (각 시점마다)
- 10 시점 역전파: $0.5^{10} ≈ 0.00098$ (거의 0)
- 20 시점 역전파: $0.5^{20} ≈ 0.00000095$ (완전히 소실)

그래디언트 폭주의 예:
- $\frac{\partial h_i}{\partial h_{i-1}} = 1.5$ (각 시점마다)
- 10 시점 역전파: $1.5^{10} ≈ 57.7$
- 20 시점 역전파: $1.5^{20} ≈ 3325$ (폭발)

### 결과

- **소실**: 장기 의존성(long-term dependency)을 학습하지 못함
- **폭주**: 학습이 불안정해지고 발산

이 문제를 해결하기 위해 **LSTM**과 **GRU**가 개발되었습니다.
